In [1]:
from aicsimageio import AICSImage
import napari
from aicsimageio.readers import CziReader
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [198]:
# VISUALIZATION

# visualize the image with napari using its numpy array
def visualize_napari(numpy_img: np.ndarray, name):
    """
    :param numpy_img: image to be visualized
    """
    with napari.gui_qt():
        viewer = napari.Viewer()
        viewer.add_image(numpy_img, name=name)


# visualize multiple images at once
def visualize_all_list_napari(numpy_img_list: np.ndarray, names):
    """
    :param numpy_img_list: list containing different images to be visualized
    """
    with napari.gui_qt():
        viewer = napari.Viewer()
        for i, img in enumerate(numpy_img_list):
            viewer.add_image(img, name=names[i])



# PREPROCESSING

# rescale and convert image 
def rescale(image: np.ndarray):
    """
    :param image: to  be rescaled
    :return: rescaled image
    """
    rescaled_image = (image - np.min(image)) / (np.max(image) - np.min(image)) * 255
    # rescaled_image = np.round(rescaled_image).astype(np.uint16)
    rescaled_image = np.round(rescaled_image).astype('uint8')
    return rescaled_image



# THRESHOLDING

# apply adaptive thresholding with Gaussian weighted sum
def adpt_g_thresholding(image: np.ndarray):
    """
    Threshold and binarize an image using adaptive thresholding using a Gaussian weighted sum

    :param image: image you want to threshold
    :return: ret: threshold value
              th: binary image
    """
    # The threshold value is a gaussian-weighted sum of the neighbourhood (here of size 25) values minus the constant C (which is set to -7)
    th = cv.adaptiveThreshold(image,255,cv.ADAPTIVE_THRESH_MEAN_C,cv.THRESH_BINARY,25,-5)
    
    return th



# POSTPROCESSING 

# remove noise
def remove_noise(image: np.ndarray):
    """
    Perform morphological opening and closing to remove noise

    :param img: image to be cleaned
    :return:    cleaned image
    """

    # define kernel for opening
    kernel = cv.getStructuringElement(cv.MORPH_RECT,(2,2))

    # perform morphological opening to remove background noise
    opening = cv.morphologyEx(image, cv.MORPH_OPEN, kernel)

    # perform morphological closing to close small holes inside foreground objects
    closing = cv.morphologyEx(opening, cv.MORPH_CLOSE, kernel)
    
    return closing


# add 2D bounding boxes to the image
def add_bounding_boxes(image, stats):
    """
    Add white rectangles around bacilli, based on conected components

    :param image: image with bacilli to be boxed
    :param coordinates:  coordinates of the center of the bacillus
    """
    image_copy = image.copy()
    for i in range(1,len(stats)):
            x = stats[i][0] - 5
            #x_max = coordinates[i][0]
            y = stats[i][1] - 5
            #y_max = coordinates[i][1]
            h=stats[i][3]
            w=stats[i][2]
            cv.rectangle(image_copy, (x, y), (x+w+10, y+h+10), (255, 0, 0), 1)
    
    return image_copy

In [85]:
# Load the image
reader = CziReader("extern_Synlab_2156_17_3_MTB.czi")

# Get whole image
smear = reader.get_image_data("MYX", C=0)

In [179]:
# save 675th tile from smear in separate variable and create a copy for future purposes
img = smear[674]

In [196]:
# rescale image to uint8
rescaled_img = rescale(img)

# apply adaptive thresholding using a Gaussian weighted sum
thd_img = adpt_g_thresholding(rescaled_img)

# remove noise from binarized image using morphological opening and closing 
cleaned_thd_img = remove_noise(thd_img)

# retrieve stats in order to add boxes around connected components, i.e., objects
num_labels, labels, stats, centroids = cv.connectedComponentsWithStats(cleaned_thd_img, connectivity=8)

# add boxes to cleaned, binarized image
masked_thd_img = add_bounding_boxes(cleaned_thd_img, stats)

# add boxes to original image
masked_img = add_bounding_boxes(img, stats)

titles = ['Original Rescaled Image', 'Binarized Image', 'Cleaned Binarized', 'Binarized w/Boxes', 'OG w/Boxes']
images = [rescaled_img, thd_img, cleaned_thd_img, masked_thd_img, masked_img]

In [197]:
# visualize
visualize_all_list_napari(images, titles)

c:\Users\idabu\.vscode\TBProject\.venv\lib\site-packages\napari\_qt\qt_event_loop.py:284: FutureWarning: 
The 'gui_qt()' context manager is deprecated.
If you are running napari from a script, please use 'napari.run()' as follows:

    import napari

    viewer = napari.Viewer()  # no prior setup needed
    # other code using the viewer...
    napari.run()

In IPython or Jupyter, 'napari.run()' is not necessary. napari will automatically
start an interactive event loop for you: 

    import napari
    viewer = napari.Viewer()  # that's it!

  warn(
